In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

See /nfs/home/genovese/thesis-wildfire-genovese/outputs/EDA_ProfileReport.html for the EDA, here the consequent preprocessing

In [ ]:
data = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/table_data_input.csv')

### 1. drop duplicates and constant variables

In [ ]:
data.drop_duplicates(inplace=True)
# data.drop(columns=['VEGETAZIONE_TIPO_Larici-cembreto su rodoreto-vaccinieto',
# 'VEGETAZIONE_INTERVENTO_Ceduazione a ceduo a sterzo'],
#                    inplace=True)

In [ ]:
# data.dropna(axis=1, how='all', inplace=True)

### 2. Check for almost constant variables

In [8]:
for c in data.columns:
    vals = data[c].value_counts().sort_values(ascending = False)
    if vals.iloc[1:].sum() <= 50:
        print(f'Column {c} has value counts: ', vals)

Column VEGETAZIONE_TIPO_Faggeta eutrofica appenninica has value counts:  VEGETAZIONE_TIPO_Faggeta eutrofica appenninica
0.000000    10825
0.126463        1
0.639343        1
0.344574        1
0.496677        1
0.301506        1
0.670335        1
0.779786        1
0.485140        1
0.397453        1
0.957758        1
0.657903        1
0.309994        1
0.271477        1
0.391189        1
0.073550        1
0.627786        1
0.623904        1
0.082605        1
0.188921        1
0.013735        1
0.185929        1
0.008073        1
0.303031        1
0.172757        1
0.239244        1
0.002598        1
0.671701        1
0.002309        1
0.151742        1
Name: count, dtype: int64
Column VEGETAZIONE_TIPO_Arbusteto di Spartium junceum
 has value counts:  VEGETAZIONE_TIPO_Arbusteto di Spartium junceum\n
0.000000    10836
0.055849        2
0.004300        2
0.048882        1
0.220489        1
0.022823        1
0.020288        1
0.000532        1
0.006135        1
0.087307        1
0.004082   

In [ ]:
# data['TOTALE_VEGETAZIONE'] = data.loc[:, [col for col in data.columns if 'VEGETAZIONE_TIPO' in col]].sum(axis=1)
# data['TOTALE_STRADE'] = data.loc[:, [col for col in data.columns if 'RTP' in col]].sum(axis=1)

In [9]:
to_drop = []

for c in data.columns:
    vals = data[c].value_counts().sort_values(ascending = False)
    if vals.iloc[1:].sum() <= 10:
        if np.max(list(vals.index)) < 0.1:
            to_drop.append(c)

In [ ]:
# data.drop(columns=to_drop, inplace=True)

### 3. Look for too high correlations

In [11]:
def find_high_corr_pairs(df, threshold=0.9):
    corr = df.corr()  # compute correlation matrix
    # Extract only upper triangle (exclude duplicates and self-correlation)
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    
    # Stack into long format and filter
    high_corr = upper.stack().loc[lambda x: x.abs() > threshold]
    # Result is a Series with MultiIndex (col1, col2) and value = correlation

    return high_corr

In [14]:
high_corr = find_high_corr_pairs(data.drop(columns='geometry'), threshold=0.95)
for s in high_corr.index:
    print(s)

('fire_id', 'target')
('analfabeti', 'alfabeti privi di titolo di studio')
('analfabeti', 'licenza di scuola elementare')
('analfabeti', 'licenza di scuola media inferiore o di avviamento professionale')
('analfabeti', 'diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS')
('analfabeti', 'diploma di tecnico superiore ITS o titolo di studio terziario di primo livello')
('analfabeti', 'titolo di studio terziario di secondo livello e dottorato di ricerca')
('analfabeti', 'titolo di studio terziario di secondo livello')
('analfabeti', 'dottorato di ricerca/diploma accademico di formazione alla ricerca')
('analfabeti', 'totale_istruzione')
('analfabeti', '9_24_nessun titolo di studio')
('analfabeti', '9_24_licenza di scuola elementare')
('analfabeti', '9_24_licenza di scuola media inferiore o di avviamento professionale')
('analfabeti', '9_24_diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) 

In [15]:
save_clean_data(data.drop(columns='geometry'), 'processed_not_imputed', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache_resampled')

File saved to: /nfs/home/genovese/thesis-wildfire-genovese/database/cache_resampled/processed_not_imputed


### 4. NA imputing
see knn_imputer.py, to do inside each kfold